<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/Organize_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rename the dataset

In [5]:
!pip install mediapipe

     |████████████████████████████████| 30.1 MB 13 kB/s 


In [6]:
import os
import cv2
import mediapipe as mp

In [2]:
dataset_raw = '/content/drive/MyDrive/Colab Notebooks/Datasets/OurDataset/Raw_Dataset'
dataset_raw_renamed = '/content/drive/MyDrive/Colab Notebooks/Datasets/OurDataset/Raw_Dataset_Renamed'


In [8]:
letters = ['A', 'B', 'C', 'D', 'E',
           'F', 'G', 'H', 'I', 'J',
           'K', 'L', 'M', 'N', 'O',
           'P', 'Q', 'R', 'S', 'T',
           'U', 'V', 'W', 'X', 'Y',
           'Z']
numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [14]:
def make_dir(path, class_obj):
  for obj in class_obj:
    path_class = os.path.join(path, obj)
    os.makedirs(path_class)

In [31]:
if os.path.isdir(os.path.join(dataset_raw_renamed, 'A')) is False:
  make_dir(dataset_raw_renamed, letters)
else:
  print('Path already exist')

In [11]:
class HandDetector():
    def __init__(self, mode=False, maxHands=2, detectionConf=0.5, trackingConf=0.5):
        # Arguments
        self.mode = mode
        self.maxHands = maxHands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        # Initialize mediapipe variables
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionConf, self.trackingConf)
        self.mpDraw = mp.solutions.drawing_utils


    def find_hands(self, src_img, draw=False):
        # Convert frame to rgb for mediapipe
        rgb = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        rgb.flags.writeable = False
        self.results = self.hands.process(rgb)

        if self.results.multi_hand_landmarks:
            x_pts, y_pts = [], []
            for handLMs in self.results.multi_hand_landmarks:
                for id, lm in enumerate(handLMs.landmark):
                    h, w, c = src_img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    x_pts.append(cx)
                    y_pts.append(cy)
                if draw:
                    self.mpDraw.draw_landmarks(src_img, handLMs, self.mpHands.HAND_CONNECTIONS)

            # Find the max and min points
            y_max, y_min, x_max, x_min = max(y_pts), min(y_pts), max(x_pts), min(x_pts)
            return True, (x_min - 100, y_max + 100), (x_max + 100, y_min - 100)

        return False, (), ()

In [12]:
detector = HandDetector()

In [19]:
def resize_image(src_img, img_size=(224, 224)):
  return cv2.resize(src_img, img_size, interpolation=cv2.INTER_CUBIC)

In [44]:
dataset_raw_letters = os.path.join(dataset_raw, 'Letters')
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
for letter in letters:
  path_class = os.path.join(dataset_raw_letters, letter)
  path_class_renamed = os.path.join(dataset_raw_renamed, letter)
  i = 0
  for item in os.listdir(path_class):
    img = cv2.imread(os.path.join(path_class, item))
    detected, pts_upper_left, pts_lower_right = detector.find_hands(img)
    if detected:
      file_name = letter + "_" + str(i) + ".jpg"
      roi = img[abs(pts_lower_right[1]):abs(pts_upper_left[1]), abs(pts_upper_left[0]):abs(pts_lower_right[0])]
      roi = resize_image(roi)
      cv2.imwrite(os.path.join(path_class_renamed, file_name), roi)
      i += 1


In [9]:
dataset_raw_letters = os.path.join(dataset_raw, 'Letters')
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
for letter in letters:
  path_class = os.path.join(dataset_raw_letters, letter)
  path_class_renamed = os.path.join(dataset_raw_renamed, letter)
  print(letter, len(os.listdir(path_class_renamed)))

A 75
B 55
C 54
D 71
E 77
F 77
G 74
H 67
I 76
J 0
K 77
L 73
M 72
N 68
O 70
P 69
Q 65
R 67
S 67
T 67
U 67
V 77
W 59
X 77
Y 80
Z 0
